## Import required Libraries and Modules

In [1]:
import os
from sentence_transformers import SentenceTransformer
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from pinecone import Pinecone, ServerlessSpec
from uuid import uuid4 
from groq import Groq
import gradio as gr
import pinecone
from dotenv import load_dotenv
from uuid import uuid4 
import requests
from langchain_core.documents import Document
load_dotenv()

# Load local embedding model (768-dim)
embedding_model = SentenceTransformer("all-mpnet-base-v2")

c:\Users\Taj\Documents\Procurement_chatbot\ollama_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Safety and Relevance Guardrail Function

In [2]:
# --- Cell 3: safety-only check ---
import re

def is_safe_and_relevant(query):
    """
    Safety-only check. Returns (True, "") if safe.
    This function DOES NOT decide relevance. Relevance is decided
    later by the main LLM (llama-3.3-70b-versatile).
    """
    if not isinstance(query, str) or not query.strip():
        return False, "Empty or invalid query."

    lowered = query.lower()

    # Simple unsafe patterns (customize as needed)
    unsafe_patterns = [
        r"\bkill\b", r"\bterror\b", r"\bself[- ]?harm\b", r"\bsuicid\b",
        r"\bsexual\b", r"\bsex\b", r"\bchild\b", r"\bexplosive\b",
        r"\bbomb\b", r"\bpoison\b", r"\bhack\b", r"\bpassword\b",
        r"\bssn\b", r"\bcredit card\b", r"\bshoot\b", r"\battack\b",
        r"\bweapon\b", r"\bdrugs?\b"
    ]

    for pat in unsafe_patterns:
        if re.search(pat, lowered):
            return False, "Your question contains unsafe content and cannot be processed."

    return True, ""


In [3]:
def ask_with_guard(question: str, retriever, llama_guard):
    # --- Guard the input ---
    if not llama_guard(question):
        return " Question blocked by Llama Guard."

    # --- Get context from retriever ---
    docs = retriever.get_relevant_documents(question)
    if not docs:
        return "I don't know based on the provided documents."

    #  Keep only your PDF sources, drop web ones
    filtered_docs = []
    for d in docs:
        meta = getattr(d, "metadata", {})
        src = str(meta.get("source", "")).lower()
        if src.endswith(".pdf"):   # only your uploaded syllabus/linear algebra
            filtered_docs.append(d)

    if not filtered_docs:
        return "I don't know based on the provided documents."

    context = "\n\n".join(
        f"[{i+1}] {d.page_content}" for i, d in enumerate(filtered_docs)
    )

    # --- Strict system prompt ---
    system_prompt = (
        "You are a STRICT retrieval-augmented assistant.\n"
        "- Only answer from the provided context.\n"
        "- If the context is insufficient, reply exactly:\n"
        "\"I don't know based on the provided documents.\"\n"
        "- Always cite sources like [1], [2].\n"
    )

    user_prompt = f"Context:\n{context}\n\nQuestion: {question}"

    # --- Call Groq LLM ---
    draft = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": user_prompt},
        ],
        temperature=0.0,
        max_tokens=700,
    ).choices[0].message.content.strip()

    # --- Guard the output ---
    if not llama_guard(draft):
        return "❌ Answer blocked by Llama Guard."

    return draft


In [4]:
# Internet Search Utility
import os
GOOGLE_API_KEY = os.getenv("GOOGLE_API_KEY")
GOOGLE_SEARCH_ENGINE_ID = os.getenv("GOOGLE_SEARCH_ENGINE_ID")

def google_search(query, num_results=5):
    """Perform a Google Custom Search and return text snippets."""
    print("Searching the internet...")  # Required for visibility

    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "key": GOOGLE_API_KEY,
        "cx": GOOGLE_SEARCH_ENGINE_ID,
        "q": query,
        "num": num_results,
    }
    
    response = requests.get(url, params=params)
    data = response.json()

    results = []
    for item in data.get("items", []):
        snippet = item.get("snippet", "")
        link = item.get("link", "")
        results.append(f"{snippet} (Source: {link})")

    return "\n".join(results) if results else "No relevant web results found."


In [5]:
# Structured Web Search + Per-Source Scoring + Rendering
# This leaves your original `google_search` untouched.
# Dependencies: GOOGLE_API_KEY / GOOGLE_SEARCH_ENGINE_ID env, `requests` imported earlier, `embedding_model` available.

from sklearn.metrics.pairwise import cosine_similarity

def google_search_structured(query: str, num_results: int = 10):
    """
    Google CSE — returns a list of dicts with title, link, snippet.
    Prints 'Searching the internet...' (as required).
    """
    print("Searching the internet...")
    url = "https://www.googleapis.com/customsearch/v1"
    params = {
        "key": os.getenv("GOOGLE_API_KEY"),
        "cx": os.getenv("GOOGLE_SEARCH_ENGINE_ID"),
        "q": query,
        "num": num_results,
    }
    resp = requests.get(url, params=params)
    data = resp.json()
    results = []
    for item in data.get("items", []):
        results.append({
            "title": item.get("title", ""),
            "link": item.get("link", ""),
            "snippet": item.get("snippet", "")
        })
    return results



def render_sources_with_scores(results: list, header: str = "Web Sources"):
    """
    Pretty-prints sources with individual scores (Title, URL, Snippet, Score).
    """
    if not results:
        return f"=== {header} ===\nNo web sources found."
    lines = [f"=== {header} (Top {len(results)}) ==="]
    for i, r in enumerate(results, 1):
        lines.append(
            f"[{i}] {r.get('title','')}\n"
            f"URL: {r.get('link','')}\n"
            f"Score: {r.get('score','0')}\n"
            f"Snippet: {r.get('snippet','')}\n"
        )
    return "\n".join(lines)


## Groq API Connection Test

In [6]:
client = Groq(
    api_key=os.environ.get("GROQ_API_KEY"),
)
question = input("What is your Question?")
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": question,
        }
    ],
    model="llama-3.3-70b-versatile",
)

print(chat_completion.choices[0].message.content)

It seems like you didn't ask a question or provide any text for me to respond to. Could you please provide more context or ask a question so I can assist you better? I'm here to help with any information or topics you're interested in.


## Pinecone Vector Database Initilization

In [7]:
# Initialize Pinecone client (connecting to existing index)
pc = Pinecone(api_key=os.getenv("PINECONE_API_KEY"))
index = pc.Index('procurement-chatbot')

# Load embedding model for queries only  
embedding_model = SentenceTransformer("all-mpnet-base-v2")

In [8]:
def get_embedding(text="None"):
    embedding = embedding_model.encode(text).tolist()
    return embedding

## Text Embedding Function

In [9]:
def count_tokens(text):
    return len(text.split())

##  Prototyping the RAG Pipeline

In [10]:
# Load embedding model
embedding_model = SentenceTransformer('all-mpnet-base-v2')

# Get the user query
user_query = input("Ask something: ")

# Convert the query to an embedding
query_embedding = embedding_model.encode(user_query).tolist()

# Search Pinecone index
top_k = 5
results = index.query(vector=query_embedding, top_k=top_k, include_metadata=True)

# Extract relevant chunks for LLM context
relevant_chunks_text = [match['metadata']['text'] for match in results['matches']]

# Combine chunks into a single context string
context = "\n\n".join(relevant_chunks_text)

# Formulate the prompt for the LLM
prompt_for_llm = f"""Based on the following context, please answer the question.
If the answer is not available in the context, state that you cannot answer from the provided information.

Context:
{context}

Question: {user_query}

Answer:"""

# Call the Groq LLM for inference
chat_completion = client.chat.completions.create(
    messages=[
        {
            "role": "user",
            "content": prompt_for_llm,
        }
    ],
    model="llama-3.3-70b-versatile", # Or preferred Groq model
    temperature=0.7,
    max_tokens=500,
)

# Print the LLM's generated response
print("\n--- LLM Generated Response ---")
print(chat_completion.choices[0].message.content)

print("\n--- Retrieved Relevant Chunks with Metadata ---")
for i, match in enumerate(results['matches']):
    chunk_text = match['metadata']['text']
    filename = match['metadata'].get('filename', 'N/A')
    page_number = match['metadata'].get('page_number', 'N/A')
    score = match['score']

    print(f"\nChunk {i+1}:")
    print(f"  Score: {score:.4f}")
    print(f"  Source File: {filename}")
    print(f"  Page Number: {page_number}")
    print(f"  Content:\n{chunk_text}")
    print("-" * 60)


--- LLM Generated Response ---
It appears you haven't provided a question for me to answer based on the given context. Please provide the question, and I'll be happy to assist you. If the answer is not available in the context, I will state that I cannot answer from the provided information.

--- Retrieved Relevant Chunks with Metadata ---

Chunk 1:
  Score: 0.2451
  Source File: 1- InnovatiCS - DS & AI Zero to Hero  Batch 22 ( Feb 22 -2025).pdf
  Page Number: 1.0
  Content:
Service
Marquis Who’s Who 2024
over 125 years
Internet 2.0 Outstanding
Leadership Award-Dubai 2022
/gid00042/gid00077/gid00083/gid00068/gid00081/gid00077/gid00064/gid00083/gid00072/gid00078/gid00077/gid00064/gid00075/gid00001/gid00034/gid00082/gid00082/gid00078/gid00066/gid00072/gid00064/gid00083/gid00072/gid00078/gid00077/gid00001/gid00078/gid00069/gid00001
------------------------------------------------------------

Chunk 2:
  Score: 0.1941
  Source File: 1- InnovatiCS - DS & AI Zero to Hero  Batch 22 ( Feb 22 

In [11]:
# ================================
#  Smart RAG Query Handler with Agentic Web Search
# ================================

# System prompt for LLM
system_prompt = """
You are an assistant that answers using the following sources in order:

1. **Primary Source:** The provided document context (RAG retrieval results)
2. **Secondary Source:** If available, additional web search results

Instructions:
- Prefer document context first. If the answer is clearly not in the documents, 
it is likely off-topic..
-Only answer questions based off of provided documents. Must be relevant to the provided documents 
- Only use the web if documents don't answer the query.
- If both sources are insufficient, respond:
"I'm sorry, I couldn't find enough relevant information to answer that."
- Follow these behavioral guidelines:
    - If unsure, admit it instead of guessing or hallucinating.
    - Only say a question is unrelated if it clearly has no connection to provided documents
    - Avoid generic or evasive responses. Be specific and clear, even if the answer is limited.
    - Do not reject questions that are vague or short; instead, ask clarifying questions if needed.
    - Assume the user may be continuing a conversation and handle follow-ups like "elaborate" or "explain more."
""".strip()


def smart_rag_query(user_query):
    """
    1. Retrieves context from RAG.
    2. LLM decides if web search is needed.
    3. Combines web results with RAG context if needed.
    4. Returns final LLM answer using system prompt.
    """

    # --- Step 1: Safety & relevance check
    is_safe, msg = is_safe_and_relevant(user_query)
    if not is_safe:
        return msg

    # --- Step 2: Retrieve context from your existing RAG pipeline
    # ⚠ Replace with your actual retrieval function if named differently
    rag_context = retrieve_context(user_query)
    context_text = "\n".join(rag_context) if isinstance(rag_context, list) else rag_context

    # --- Step 3: Ask LLM if web search is needed
    decision_prompt = f"""
    You are a decision-making agent.
    User Query: {user_query}
    RAG Context (truncated): {context_text[:1000]}
    
    Decide if the retrieved document context is sufficient to answer.
    Reply with exactly one word: "SEARCH" or "NO SEARCH"
    """.strip()

    decision = llm_generate_response(decision_prompt).strip().upper()
    print(f"[DEBUG] LLM Decision: {decision}")  # For testing

    # --- Step 4: Perform web search if LLM decides so
    if "SEARCH" in decision:
        web_results = google_search(user_query)
        combined_context = f"{context_text}\n\nAdditional Web Info:\n{web_results}"
    else:
        combined_context = context_text

    # --- Step 5: Generate final response using LLM with system prompt
    final_prompt = f"""
    {system_prompt}

    Document & Web Context:
    {combined_context}

    User Query: {user_query}
    """.strip()

    final_answer = llm_generate_response(final_prompt)
    return final_answer.strip()


## RAG Function with Integrated Guardrails

In [12]:
def get_answer_with_guardrails(query):
    """
    Retrieve top-k docs, show scores + previews, then ask
    llama-3.3-70b-versatile to DECIDE relevance and answer only if supported.
    NO web fallback. If out-of-scope, it refuses.
    """
    # 1) Embed query
    query_embedding = embedding_model.encode([query])[0].tolist()

    # 2) Query Pinecone
    results = index.query(vector=query_embedding, top_k=15, include_metadata=True)
    matches = results.get("matches", []) if isinstance(results, dict) else getattr(results, "matches", []) or []

    # 3) Show top-10 with score + preview
    print("📌 Retrieved Sources (top 10):")
    sources_list = []
    for i, m in enumerate(matches[:10], start=1):
        score = m.get("score", 0.0)
        meta = m.get("metadata", {}) or {}
        filename = meta.get("filename", meta.get("source", f"chunk_{i}"))
        text = meta.get("text", meta.get("content", ""))
        words = text.split()
        preview = " ".join(words[:25]) + ("..." if len(words) > 25 else "")
        print(f"\n[{i}] file={filename}  vec_score={float(score):.4f}")
        print(f"Preview: {preview}")
        sources_list.append({"index": i, "score": float(score), "filename": filename, "text": text})

    if not sources_list:
        return "Sorry, I couldn't find any documents to answer that question."

    # 4) Build sources for LLM
    sources_text = "\n\n".join(
        [f"[{s['index']}] (score={s['score']:.4f}) {s['filename']}\n{s['text']}" for s in sources_list]
    )

    # 5) Strict prompt
    judge_prompt = f"""
You are a strict Retrieval-Augmented Generation (RAG) assistant.

RULES:
1. Only answer using the SOURCES below.
2. If the QUESTION cannot be answered from these sources, reply with exactly:
   This question is out of scope of the provided documents.
3. Do not use outside knowledge or web search.

QUESTION:
{query}

SOURCES:
{sources_text}
"""

    # 6) One LLM call
    completion = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[
            {"role": "system", "content": "You must refuse if the question is out of scope of provided docs."},
            {"role": "user", "content": judge_prompt}
        ],
        temperature=0.0
    )

    resp_text = completion.choices[0].message.content.strip()

    # 7) Normalize refusal
    if "out of scope" in resp_text.lower():
        return "This question is out of scope of the provided documents."

    return resp_text

# Hook into Gradio
standalone_rag = get_answer_with_guardrails


## Gradio Chatbot User Interface

In [13]:
def is_general_knowledge(query):
    """Check if it's a general knowledge question (not doc-related)."""
    keywords = ["who", "what", "when", "where", "why", "how"]
    return any(word in query.lower() for word in keywords)


def web_search(query):
    """Placeholder for web search logic (tooling feature goes here)."""
    return f"[Web Search Result Placeholder for: '{query}']"

def format_sources(results):
    """Format source metadata into a readable string box."""
    source_box = ""
    for match in results['matches']:
        doc = match['metadata'].get('source', 'Unknown Doc')
        page = match['metadata'].get('page', 'N/A')
        source_box += f"📄 **{doc}** – Page {page}\n"
    return source_box


def agentic_chat_with_guardrails(query, history):
    """Main chat function with decision-making guardrails."""

    # Step 1: Embed the query
    query_embedding = embed.embed_query(query)

    # Step 2: Run similarity search on uploaded docs
    results = index.query(
        vector=query_embedding,
        top_k=10,
        include_metadata=True
    )

    # Step 3: Agentic logic - choose RAG, Web, or Reject
    top_score = results['matches'][0]['score'] if results['matches'] else 0

    if top_score > 0.75:
        # RAG path (use docs)
        context = "\n\n".join([m["metadata"]["text"] for m in results["matches"]])
        source_info = format_sources(results)

        prompt = f"""Use the following context to answer the question.
        If it cannot be answered, say "I don't know."

        Context:
        {context}

        Question:
        {query}
        """

        response = client.chat.completions.create(
            model="llama3-70b-8192",
            messages=[
                {"role": "system", "content": "You are a helpful assistant."},
                {"role": "user", "content": prompt}
            ]
        )

        answer = response.choices[0].message.content.strip()
        return answer, source_info

    elif is_general_knowledge(query):
        # Web search path
        result = web_search(query)
        return result, "🌐 Web Search Results"

    else:
        # Reject unrelated queries
        return "❌ Sorry, I can only answer questions related to the uploaded documents or general knowledge.", ""


In [ ]:
import gradio as gr
from sklearn.metrics.pairwise import cosine_similarity

# Keep your existing response quality function (unchanged)
def evaluate_response_quality(response, source_chunks):
    if not source_chunks or not response:
        return 0.0
    response_embed = embedding_model.encode([response])
    source_embeds = embedding_model.encode(source_chunks)
    scores = cosine_similarity(response_embed, source_embeds)
    print("scores", scores)
    return float(scores.max())

# Updated strict system prompt
strict_system_prompt = """
You are a STRICT document-based assistant. You can ONLY answer questions that are directly related to the provided document context.

STRICT RULES:
1. If the question cannot be answered using the provided document context, respond with exactly: "This question is outside the scope of the provided documents."
2. Do not use any general knowledge or external information
3. Do not answer questions about general topics (like food, weather, celebrities, etc.) even if you have the knowledge
4. Only answer if the information is explicitly present in the documents
5. The documents appear to be about Data Science, AI, and technical training - only answer questions related to these topics if they can be found in the context

Examples of what to REFUSE:
- "What is a burger?" → "This question is outside the scope of the provided documents."
- "Who is the president?" → "This question is outside the scope of the provided documents."
- "How to cook pasta?" → "This question is outside the scope of the provided documents."

Examples of what to ANSWER (only if found in documents):
- Questions about the training program mentioned in documents
- Questions about specific content covered in the curriculum
- Questions about instructors or course details mentioned in the documents
""".strip()

def standalone_rag(query):
    # 1) Embed query and search Pinecone (your existing vector DB) - NOW 10 SOURCES
    query_embedding = embedding_model.encode(query).tolist()
    results = index.query(vector=query_embedding, top_k=10, include_metadata=True)

    # 2) Build RAG doc context + doc source list - NOW WITH INDIVIDUAL SCORES
    context_chunks = []
    doc_sources = []
    for i, match in enumerate(results.get("matches", []), 1):
        meta = match.get("metadata", {}) or {}
        text = meta.get("text", "")
        doc = meta.get("filename", "Unknown Doc")
        page = meta.get("page_number", "N/A")
        score = match.get("score", 0.0)
        
        context_chunks.append(text)
        doc_sources.append(f"[D{i}] {doc} - Page {page} (Score: {score:.4f})")

    rag_context = "\n\n".join(context_chunks) if context_chunks else "No relevant document context found."

    # 3) Check relevance threshold - if top score is too low, likely off-topic
    top_score = results.get("matches", [{}])[0].get("score", 0.0) if results.get("matches") else 0.0
    
    if top_score < 0.3:  # Adjust this threshold as needed
        return "This question is outside the scope of the provided documents.", "No relevant sources found.", "0.000"

    # 4) LLM decision: do we need internet? (you already had this — preserving and using it)
    decision_prompt = f"""
You are an agent deciding if a web search is needed.
Query: {query}
Document Context (first 500 chars):
{rag_context[:500]}

The documents are about Data Science, AI, and technical training.

Reply exactly:
SEARCH      -> if the document context is insufficient but the question IS related to the document topics
NO SEARCH   -> if the document context fully answers the query
REFUSE      -> if the question is completely unrelated to the document topics (like asking about food, celebrities, general knowledge, etc.)
""".strip()

    try:
        decision_response = client.chat.completions.create(
            model="llama-3.3-70b-versatile",
            messages=[{"role": "user", "content": decision_prompt}],
            temperature=0,
            max_tokens=20
        )
        decision = decision_response.choices[0].message.content.strip().upper()
    except Exception as e:
        print("Decision call failed; defaulting to REFUSE for safety. Error:", e)
        decision = "REFUSE"

    # 5) If REFUSE, return early
    if "REFUSE" in decision:
        return "This question is outside the scope of the provided documents.", "Question deemed off-topic.", "0.000"

    # 6) If SEARCH -> fetch ≥10 web results, score, and prepare combined context
    web_used = (decision == "SEARCH")
    web_results_scored = []
    web_context = ""
    web_sources_display = ""

    if web_used:
        # at least 10 sources as required
        raw_web = google_search_structured(query, num_results=10)
        web_results_scored = score_sources_embeddings(query, raw_web)
        web_sources_display = render_sources_with_scores(web_results_scored, header="Web Sources")

        # Build a compact context block from top web snippets (limit tokens by truncating)
        top_snippets = [r.get("snippet", "") for r in web_results_scored]
        web_context = "\n\n".join(top_snippets)

    # 7) Combine contexts (Docs + Optional Web)
    combined_context = rag_context
    if web_used and web_context:
        combined_context += "\n\n---\n[Web Context]\n" + web_context

    # 8) Final prompt to LLM with STRICT system prompt
    final_prompt = f"""
{strict_system_prompt}

Context (Docs + Optional Web):
{combined_context}

User Query: {query}

IMPORTANT: Before answering, determine if this query can be answered from the provided context. If not, respond with exactly: "This question is outside the scope of the provided documents."

Rules for valid answers:
- If you used web sources, cite them by index like [W1], [W2] that correspond to the numbered list in "Web Sources".
- If you used doc sources, cite them like [D1], [D2] that correspond to the order they appear in the doc source list.
- Be concise, accurate, and do not hallucinate.
""".strip()

    final_response = client.chat.completions.create(
        model="llama-3.3-70b-versatile",
        messages=[{"role": "user", "content": final_prompt}],
        temperature=0.1,  # Lower temperature for more consistent refusals
        max_tokens=500
    )
    answer = final_response.choices[0].message.content

    # 9) Double-check for off-topic responses - if the LLM still answered inappropriately
    if "This question is outside the scope" not in answer and top_score < 0.5:
        # Additional safety check
        safety_check_prompt = f"""
Query: {query}
Response: {answer}
Documents are about: Data Science, AI, technical training

Is this response appropriate for documents about Data Science/AI training?
Reply: YES or NO
""".strip()
        
        try:
            safety_response = client.chat.completions.create(
                model="llama-3.3-70b-versatile",
                messages=[{"role": "user", "content": safety_check_prompt}],
                temperature=0,
                max_tokens=5
            )
            is_appropriate = safety_response.choices[0].message.content.strip().upper()
            
            if "NO" in is_appropriate:
                return "This question is outside the scope of the provided documents.", "Safety check triggered.", "0.000"
        except Exception as e:
            print("Safety check failed:", e)

    # 10) Build the Sources panel output: WITH INDIVIDUAL SCORES
    sources_sections = []
    if doc_sources:
        sources_sections.append("=== Document Sources (10) ===\n" + "\n".join(doc_sources))
    if web_used:
        sources_sections.append(web_sources_display)
    sources_text = "\n\n".join(sources_sections) if sources_sections else "No sources."

    # 11) Compute response quality against all available chunks (docs + top web snippets)
    source_chunks_for_quality = []
    source_chunks_for_quality.extend(context_chunks)
    if web_used and web_results_scored:
        source_chunks_for_quality.extend([r.get("snippet", "") for r in web_results_scored])
    quality = evaluate_response_quality(answer, source_chunks_for_quality)
    quality_str = f"{quality:.3f}"

    return answer, sources_text, quality_str

# === Gradio UI (preserving your layout) ===
with gr.Blocks() as demo:
    gr.Markdown("# RAG System")
    gr.Markdown("This assistant will only answer questions related to the provided documents (Data Science & AI training materials).")
    
    with gr.Row():
        with gr.Column(scale=2):
            answer_output = gr.Textbox(label="Assistant", lines=15)
        with gr.Column(scale=1):
            source_output = gr.Textbox(label="Sources", lines=10.5)
            quality_score = gr.Textbox(label="Response Quality Score", lines=1)

    with gr.Row():
        user_input = gr.Textbox(label="Ask a Question", lines=1, show_label=True)
        submit = gr.Button("Submit")

    def on_submit(query):
        answer, sources, score = standalone_rag(query)
        return answer, sources, score, ""

    submit.click(fn=on_submit, inputs=user_input, outputs=[answer_output, source_output, quality_score, user_input])
    user_input.submit(fn=on_submit, inputs=user_input, outputs=[answer_output, source_output, quality_score, user_input])

demo.launch()
#

* Running on local URL:  http://127.0.0.1:7860
* To create a public link, set `share=True` in `launch()`.
